# 

# Peak shaving vs TCIPC analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.interpolate import griddata
from weis.visualization.utils import load_OMsql_multi

plt.style.use("torque.mplstyle")

%matplotlib widget

In [ ]:
# Define the logs to load and give them labels.
logs_to_load = {
    "Free yaw": "../../../data/torque/ps_vs_tcipc/log_ps_vs_tcipc.sql*",
    "Zero yaw": "../../../data/torque/ps_vs_tcipc_zero_yaw/log_ps_vs_tcipc_zero_yaw.sql*",
}

# Load all datasets
all_data_dicts = {}
for log_name, log_fmt in logs_to_load.items():
    all_data_dicts[log_name] = load_OMsql_multi(log_fmt)
    print(f"Loaded {log_name}: {all_data_dicts[log_name].keys()}")

In [ ]:
# Let's define how we load, scale, and label the data, then make a dataframe.
all_outputs = {
    "rank": {
        "key": "rank",
        "scaling": lambda x: x,
        "label": "rank",
    },
    "iter": {
        "key": "iter",
        "scaling": lambda x: x,
        "label": "iter",
    },
    # ROSCO variables.
    "TCIPC_MaxTipDeflection": {
        "key": "tune_rosco_ivc.TCIPC_MaxTipDeflection",
        "scaling": lambda x: x,
        "label": "TCIPC Max Tip Deflection (m)",
    },
    "ps_percent": {
        "key": "tune_rosco_ivc.ps_percent",
        "scaling": lambda x: x,
        "label": "Peak Shaving (-)",
    },
    "TCIPC_nHarmonics": {
        "key": "tune_rosco_ivc.TCIPC_nHarmonics",
        "scaling": lambda x: x,
        "label": "Number of harmonics",
    },
    "TCIPC_ZeroYawDeflection": {
        "key": "tune_rosco_ivc.TCIPC_ZeroYawDeflection",
        "scaling": lambda x: x,
        "label": "Zero yaw deflection",
    },
    # Objectives / responses
    "aep": {
        "key": "aeroelastic.AEP",
        "scaling": lambda x: 1e-6 * x,
        "label": "AEP (GWh)",
    },
    "max_TipDxc_towerPassing": {
        "key": "aeroelastic.max_TipDxc_towerPassing",
        "scaling": lambda x: x,
        "label": "Max TipDxc Tower Passing (m)",
    },
    "tower_clearance": {
        "key": "aeroelastic.max_TipDxc_towerPassing",
        "scaling": lambda x: 30 - x,
        "label": "Tower clearance (m)",
    },
    "TCIPC_amplitude_at_max_deflection": {
        "key": "aeroelastic.TCIPC_amplitude_at_max_deflection",
        "scaling": lambda x: x,
        "label": "TCIPC amplitude (deg)",
    },
    "avg_pitch_travel": {
        "key": "aeroelastic.avg_pitch_travel",
        "scaling": lambda x: x,
        "label": "Avg Pitch Travel (deg)",
    },
    "DEL_RootMyb": {
        "key": "aeroelastic.DEL_RootMyb",
        "scaling": lambda x: x / 1000,
        "label": "DEL Root Myb (MNm)",
    },
    "max_TwrBsMyt": {
        "key": "aeroelastic.max_TwrBsMyt",
        "scaling": lambda x: x / 1000,
        "label": "Max Tower Base Myt (MNm)",
    },
}

# Build dataframe from mapping for each log
labels = {short: info["label"] for short, info in all_outputs.items()}
all_dfs = []

for log_name, data_dict in all_data_dicts.items():
    df_dict = {}
    for short_label, info in all_outputs.items():
        data = data_dict[info["key"]]
        scaled_data = list(map(info["scaling"], data))
        df_dict[short_label] = scaled_data

    df_temp = pd.DataFrame(df_dict)
    df_temp["log_name"] = log_name  # Add identifier column
    all_dfs.append(df_temp)

# Combine all dataframes
df = pd.concat(all_dfs, ignore_index=True)
print(f"Combined dataframe shape: {df.shape}")
df.head()

## Data exploration

In [ ]:
# Make a plot of the distribution of our design variables.
plt.figure()
sns.scatterplot(
    df,
    x="TCIPC_MaxTipDeflection",
    y="ps_percent",
    style="log_name",
)
plt.show()

In [ ]:
# Plot several outputs/objectives as a function of the design variables.
# Output variables to plot.
outputs = [
    "aep",
    "tower_clearance",
    "avg_pitch_travel",
    "TCIPC_amplitude_at_max_deflection",
    "DEL_RootMyb",
    "max_TwrBsMyt",
]

fig, axs = plt.subplots(len(logs_to_load), len(outputs), figsize=(15, 5))

for i, log_name in enumerate(logs_to_load.keys()):
    for j, output in enumerate(outputs):
        scatter = axs[i, j].scatter(
            df[df["log_name"] == log_name]["ps_percent"],
            df[df["log_name"] == log_name]["TCIPC_MaxTipDeflection"],
            c=df[df["log_name"] == log_name][output],
        )

        axs[i, j].set_xlabel("Peak shaving (-)")
        axs[i, j].set_ylabel("TCIPC max tip deflection (m)")
        axs[i, j].set_title(output)
        plt.colorbar(scatter, ax=axs[i, j])

plt.tight_layout()
plt.show()

In [ ]:
# Get an idea of the trade-off between AEP and tower clearance.
fig, ax = plt.subplots()

df.sort_values("aep", inplace=True)


sns.lineplot(
    data=df[df["TCIPC_MaxTipDeflection"] == 0.0],
    x="aep",
    y="tower_clearance",
    hue="log_name",
)
sns.lineplot(
    data=df[df["TCIPC_MaxTipDeflection"] == 20.0],
    x="aep",
    y="tower_clearance",
    hue="log_name",
    palette="deep",
)

## Data interpolation

In [ ]:
# Define the bounds of our design space for interpolation.
ps_min, ps_max = (
    df["ps_percent"].min(),
    df["ps_percent"].max(),
)
tip_min, tip_max = (
    df["TCIPC_MaxTipDeflection"].min(),
    df["TCIPC_MaxTipDeflection"].max(),
)

# Create a regular grid for interpolation to enable smooth contour plots.
n_points = 50
ps_grid = np.linspace(ps_min, ps_max, n_points)
tip_grid = np.linspace(tip_min, tip_max, n_points)
ps_percent_grid, tcipc_reference_grid = np.meshgrid(ps_grid, tip_grid)

# Interpolate each output variable on the grid for each log.
# We store the results in a nested dictionary for easy access when plotting.
interpolated_data = {}

for log_name in logs_to_load.keys():
    interpolated_data[log_name] = {}
    df_log = df[df["log_name"] == log_name]

    # Extract the independent variables as points for interpolation.
    points = df_log[["ps_percent", "TCIPC_MaxTipDeflection"]].values

    for output in outputs:
        # Extract the dependent variable values.
        values = df_log[output].values

        # Interpolate using linear method, which works well for scattered data.
        grid_values = griddata(
            points,
            values,
            (ps_percent_grid, tcipc_reference_grid),
            method="cubic",
        )

        interpolated_data[log_name][output] = grid_values

print(f"Interpolated {len(outputs)} outputs for {len(logs_to_load)} datasets")
print(f"Grid shape: {ps_percent_grid.shape}")

In [ ]:
# Plot several outputs/objectives as a function of the design variables.
# Output variables to plot.
outputs = [
    "aep",
    "tower_clearance",
    "avg_pitch_travel",
    "TCIPC_amplitude_at_max_deflection",
    "DEL_RootMyb",
    "max_TwrBsMyt",
]
clims = [
    [70, 95],
    [10, 30],
    [0, 2],
    [0, 5],
    [15, 50],
    [150, 400],
]

fig, axs = plt.subplots(len(logs_to_load), len(outputs), figsize=(14, 5))

for i, log_name in enumerate(logs_to_load.keys()):
    print(log_name)
    for j, output in enumerate(outputs):
        levels = np.linspace(clims[j][0], clims[j][1], 9)
        scatter = axs[i, j].contourf(
            ps_percent_grid,
            tcipc_reference_grid,
            interpolated_data[log_name][output],
            levels=levels,
            vmin=clims[j][0],
            vmax=clims[j][1],
        )

        axs[i, j].set_xlabel("Peak shaving (-)")
        axs[i, j].set_ylabel("TCIPC max tip deflection (m)")
        axs[i, j].set_title(output)
        plt.colorbar(scatter, ax=axs[i, j])

plt.tight_layout()

## Plots for Torque

In [ ]:
# TODO: I don't really know yet what I would like to show.

## Optimization

In [ ]:
from pymoo.core.problem import Problem, ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.moead import MOEAD
from pymoo.algorithms.moo.ctaea import CTAEA
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.termination import get_termination
from pymoo.optimize import minimize
from scipy.interpolate import CloughTocher2DInterpolator


# Create separate interpolators for each log
interpolators = {}
outputs = [
    "aep",
    "tower_clearance",
    "avg_pitch_travel",
    "TCIPC_amplitude_at_max_deflection",
    "DEL_RootMyb",
    "max_TwrBsMyt",
]
signs = [-1, -1, +1, +1, +1, +1]
for log_name in all_data_dicts.keys():
    df_log = df[df["log_name"] == log_name]
    xy = df_log[["ps_percent", "TCIPC_MaxTipDeflection"]].values

    interpolators[log_name] = {}

    for j, output in enumerate(outputs):
        interpolators[log_name][output] = CloughTocher2DInterpolator(
            xy, signs[j] * df_log[output].values
        )


class TuningProblem(ElementwiseProblem):
    def __init__(self, objective_functions, inequaltiy_constraints, xl=None, xu=None):
        # Set some defaults.
        if xl is None:
            xl = np.array([0.5, 0.0])
        if xu is None:
            xu = np.array([1.0, 20.0])

        super().__init__(
            n_var=2,
            n_obj=len(objective_functions),
            n_ieq_constr=len(inequaltiy_constraints),
            xl=xl,
            xu=xu,
        )

        self.objective_functions = objective_functions
        self.inequaltiy_constraints = inequaltiy_constraints

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = [f(*x) for f in self.objective_functions]
        out["G"] = [g(*x) for g in self.inequaltiy_constraints]


# Create problems for baseline, free yaw, and zero yaw deflection
# Baseline: TCIPC_MaxTipDeflection=20 (TCIPC disabled), ps_percent is free
# Use Free yaw data for baseline (could use either dataset)
DEL_RootMyb_baseline = interpolators["Free yaw"]["DEL_RootMyb"](0.8, 20)
problem_baseline = TuningProblem(
    [interpolators["Free yaw"]["aep"], interpolators["Free yaw"]["tower_clearance"]],
    [
        lambda x, y: interpolators["Free yaw"]["DEL_RootMyb"](x, y)
        - DEL_RootMyb_baseline
    ],
    xl=np.array([0.5, 20.0]),
    xu=np.array([1.0, 20.0]),  # Fixed TCIPC, free ps_percent
)

# Free yaw: full range
problem_free_yaw = TuningProblem(
    [interpolators["Free yaw"]["aep"], interpolators["Free yaw"]["tower_clearance"]],
    [
        lambda x, y: interpolators["Free yaw"]["DEL_RootMyb"](x, y)
        - DEL_RootMyb_baseline
    ],
)

# Zero yaw deflection: full range
problem_zero_yaw = TuningProblem(
    [
        interpolators["Zero yaw"]["aep"],
        interpolators["Zero yaw"]["tower_clearance"],
    ],
    [
        lambda x, y: interpolators["Zero yaw"]["DEL_RootMyb"](x, y)
        - DEL_RootMyb_baseline
    ],
)

algorithm = NSGA2(
    pop_size=150,
    n_offsprings=10,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True,
)
# ref_dirs = get_reference_directions("uniform", 2, n_partitions=50)
# algorithm = MOEAD(
#     ref_dirs,
#     n_neighbors=50,
#     prob_neighbor_mating=0.7,
# )
# algorithm = CTAEA(ref_dirs)  # I found this one to work very well.


termination = get_termination("n_gen", 200)

# Run optimization for all three cases
result_baseline = minimize(
    problem_baseline, algorithm, termination, seed=1, save_history=True, verbose=True
)
result_free_yaw = minimize(
    problem_free_yaw, algorithm, termination, seed=1, save_history=True, verbose=True
)
result_zero_yaw = minimize(
    problem_zero_yaw, algorithm, termination, seed=1, save_history=True, verbose=True
)

In [ ]:
# Analyze convergence for all three cases
from pymoo.indicators.hv import Hypervolume


# Function to calculate hypervolume history for a result
def calculate_hv_history(result):
    n_evals = []
    hist_F = []
    hist_cv = []
    hist_cv_avg = []

    for algo in result.history:
        n_evals.append(algo.evaluator.n_eval)
        opt = algo.opt
        hist_cv.append(opt.get("CV").min())
        hist_cv_avg.append(algo.pop.get("CV").mean())
        feas = np.where(opt.get("feasible"))[0]
        hist_F.append(opt.get("F")[feas])

    approx_ideal = result.F.min(axis=0)
    approx_nadir = result.F.max(axis=0)
    ref_point = approx_nadir

    metric = Hypervolume(
        ref_point=ref_point,
        norm_ref_point=False,
        zero_to_one=False,
        ideal=approx_ideal,
        nadir=approx_nadir,
    )

    hv = [metric.do(_F) for _F in hist_F]
    return hv, n_evals


# Calculate hypervolume for all cases
hv_baseline, n_evals_baseline = calculate_hv_history(result_baseline)
hv_free_yaw, n_evals_free_yaw = calculate_hv_history(result_free_yaw)
hv_zero_yaw, n_evals_zero_yaw = calculate_hv_history(result_zero_yaw)

# Plot convergence for all three cases
plt.figure(figsize=(7, 5))
plt.plot(hv_baseline, "o-", lw=1.5, label="Baseline", markersize=4)
plt.plot(hv_free_yaw, "s-", lw=1.5, label="Free Yaw", markersize=4)
plt.plot(hv_zero_yaw, "^-", lw=1.5, label="Zero Yaw", markersize=4)
plt.title("Convergence Comparison")
plt.xlabel("Generation")
plt.ylabel("Hypervolume")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Pareto front comparison - all three cases
pareto_baseline = -result_baseline.F[result_baseline.F[:, 0].argsort()]  # type: ignore
pareto_free_yaw = -result_free_yaw.F[result_free_yaw.F[:, 0].argsort()]  # type: ignore
pareto_zero_yaw = -result_zero_yaw.F[result_zero_yaw.F[:, 0].argsort()]  # type: ignore

plt.figure(figsize=(7, 5))
plt.plot(
    *np.transpose(pareto_baseline),
    "o-",
    label="Baseline",
    linewidth=2,
    markersize=8,
)
plt.plot(
    *np.transpose(pareto_free_yaw),
    "s-",
    label="Free Yaw",
    linewidth=2,
    markersize=6,
)
plt.plot(
    *np.transpose(pareto_zero_yaw),
    "^-",
    label="Zero Yaw",
    linewidth=2,
    markersize=6,
)
plt.legend()
plt.xlabel("AEP (GWh)")
plt.ylabel("Tower clearance (m)")
plt.title("Pareto Front Comparison")
plt.grid(alpha=0.3)
plt.tight_layout()

In [ ]:
# Visualize design space for all three cases
xl, xu = problem_free_yaw.bounds()
plt.figure(figsize=(7, 5))
plt.scatter(
    result_baseline.X[:, 0],
    result_baseline.X[:, 1],
    s=50,
    # marker="*",
    # c="red",
    label="Baseline",
    zorder=3,
)
plt.scatter(
    result_free_yaw.X[:, 0],
    result_free_yaw.X[:, 1],
    s=30,
    # facecolors="none",
    # edgecolors="blue",
    label="Free Yaw",
)
plt.scatter(
    result_zero_yaw.X[:, 0],
    result_zero_yaw.X[:, 1],
    s=30,
    # facecolors="none",
    # edgecolors="green",
    label="Zero Yaw",
)
plt.xlim(xl[0], xu[0])
plt.ylim(xl[1], xu[1])
plt.xlabel("Peak Shaving (%)")
plt.ylabel("TCIPC Max Tip Deflection (m)")
plt.title("Design Space - All Cases")
plt.legend()
plt.grid(alpha=0.3)